In [1]:
from astropy.io import fits
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import subprocess
from sklearn.cluster import DBSCAN
from analyzeImage import analyzeImage
import keras
from keras.models import load_model
from __future__ import print_function

Using TensorFlow backend.


In [3]:
gpu_code_path = "../code/gpu/"
real_image_name = "chip_7"
real_image_path = "../../HITS/trimmed_01" #gpu_code_path+"images/"+real_image_name
psi_image_path = gpu_code_path+"output-images/psi"
phi_image_path = gpu_code_path+"output-images/phi"

In [200]:
def execute(cmd):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line 
    popen.stdout.close()
    return_code = popen.wait()
    if return_code:
        raise subprocess.CalledProcessError(return_code, cmd)

In [306]:
paramsFile = open('../code/gpu/debug/parameters.config', 'w')
paramsFile.write(
"""Debug ................ : 1
Image Count .......... : 5
Generate Images ...... : 0
Image Width .......... : 500
Image Height ......... : 500
PSF Sigma ............ : 1.0
Object Brightness .... : 360.0
Object Initial x ..... : 320.0
Object Initial y ..... : 180.0
Velocity x ........... : -1.3
Velocity y ........... : -0.2
Background Level ..... : 1024.0
Background Sigma ..... : 32.0
Mask Threshold ....... : 40.0
Mask Penalty ......... : 15.0
Angles to Search ..... : 80
Velocities to Search . : 20
Minimum Velocity ..... : 300
Maximum Velocity ..... : 450
Write to file ........ : 1
Source Images Path ... : ../../{source}/
Psi Images Path ...... : ../../{psi}/
Phi Images Path....... : ../../{phi}/
""".format( source=real_image_path, psi=psi_image_path, phi=phi_image_path ))
paramsFile.close()

In [202]:
popen = subprocess.Popen( "./clearImages.sh", stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE)
popen.wait()
output = popen.stderr.read()
output += popen.stdout.read()
print( output)

In [203]:
for path in execute("./search.sh"):
    print(path, end="")

Image Count ..........  : 5
Generate Images ......  : 0
Image Width ..........  : 500
Image Height .........  : 500
PSF Sigma ............  : 1.0
Object Brightness ....  : 360.0
Object Initial x .....  : 320.0
Object Initial y .....  : 180.0
Velocity x ...........  : -1.3
Velocity y ...........  : -0.2
Background Level .....  : 1024.0
Background Sigma .....  : 32.0
Mask Threshold .......  : 40.0
Mask Penalty .........  : -0.012
Angles to Search .....  : 120
Velocities to Search .  : 40
Minimum Velocity .....  : 320
Maximum Velocity .....  : 550
Write to file ........  : 1
Source Images Path ...  : ../../../code/gpu/images/chip_7/
Psi Images Path ......  : ../../../code/gpu/output-images/psi/
Phi Images Path.......  : ../../../code/gpu/output-images/phi/
Using Kernel Size 5X5
| 0.004 | 0.015 | 0.023 | 0.015 | 0.004 | 
 ---------------------------------------
| 0.015 | 0.058 | 0.093 | 0.058 | 0.015 | 
 ---------------------------------------
| 0.023 | 0.093 | 0.147 | 0.093 | 0.023 | 
 --

In [2]:
raw_results = np.genfromtxt('../data/results/HITS1.txt', names=True)

In [6]:
image_mjd = []

for filename in sorted(os.listdir(real_image_path)):
    hdulist = fits.open(os.path.join(real_image_path, filename))
    image_mjd.append(hdulist[0].header['MJD'])

image_mjd = np.array(image_mjd)
image_times = image_mjd - image_mjd[0]
#image_times*=24.

Load Psi Images

In [27]:
hdulist = fits.open(os.path.join(psi_image_path, os.listdir(psi_image_path)[0]))
num_images = len(os.listdir(psi_image_path))
image_shape = np.shape(hdulist[0].data)
im_psi_array = np.zeros((num_images, image_shape[0], image_shape[1]))

for idx, filename in list(enumerate(sorted(os.listdir(psi_image_path)))):

    #print (str('Loaded ' + filename))

    image_file = os.path.join(psi_image_path, filename)
    hdulist = fits.open(image_file)
    im_psi_array[idx] = hdulist[0].data#*mask


Load Phi Images (for potentially making psi/phi stamps)

In [8]:
hdulist = fits.open(os.path.join(phi_image_path, os.listdir(phi_image_path)[0]))
num_images = len(os.listdir(phi_image_path))
image_shape = np.shape(hdulist[0].data)
im_phi_array = np.zeros((num_images, image_shape[0], image_shape[1]))

for idx, filename in list(enumerate(sorted(os.listdir(phi_image_path)))):

   # print (str('Loaded ' + filename))

    image_file = os.path.join(phi_image_path, filename)
    hdulist = fits.open(image_file)
    im_phi_array[idx] = hdulist[0].data#*mask


In [9]:
hdulist = fits.open(os.path.join(real_image_path, os.listdir(real_image_path)[0]))
num_images = len(os.listdir(real_image_path))
image_shape = np.shape(hdulist[1].data)
im_array = np.zeros((num_images, image_shape[0], image_shape[1]))


for idx, filename in list(enumerate(sorted(os.listdir(real_image_path)))):

   # print( str('Loaded ' + filename))

    image_file = os.path.join(real_image_path, filename)
    hdulist = fits.open(image_file)
    im_array[idx] = hdulist[1].data#*mask


In [10]:
ai = analyzeImage()

In [250]:
model = load_model('../data/kbmod_model.h5')

In [18]:
results = raw_results[0:8000]
results

array([(1969.0, 2838.0, 0.0, 0.0, 5.751, 0.0, 3.821, 214.793),
       (1973.0, 2838.0, 0.0, 0.0, 4.853, -0.511, 3.807, 214.739),
       (1971.0, 2838.0, 0.0, 0.0, 5.378, 0.0, 3.765, 206.783), ...,
       (905.0, 8.0, 0.0, 0.0, 4.066, -4.067, 1.045, 55.18),
       (1725.0, 14.0, 0.0, 0.0, -4.367, -3.931, 1.045, 70.521),
       (1982.0, 1468.0, 0.0, 0.0, 4.773, 1.015, 1.045, 64.687)], 
      dtype=[('t0_x', '<f8'), ('t0_y', '<f8'), ('theta_par', '<f8'), ('theta_perp', '<f8'), ('v_x', '<f8'), ('v_y', '<f8'), ('likelihood', '<f8'), ('est_flux', '<f8')])

In [301]:
filtered_results = ai.filter_results(im_array, results, image_times, model, chunk_size=2000)

Finished chunk 1 of 1


In [302]:
print( len(filtered_results) )
filtered_results

361


array([(632.0, 1982.0, 0.0, 0.0, -27.628, -439.132, 3.274, 3490.778),
       (633.0, 1942.0, 0.0, 0.0, -37.6, -297.634, 3.277, 3525.271),
       (633.0, 2259.0, 0.0, 0.0, -48.253, -381.964, 3.28, 3492.125),
       (633.0, 2257.0, 0.0, 0.0, -46.373, -367.082, 3.281, 3849.504),
       (633.0, 2258.0, 0.0, 0.0, -46.373, -367.082, 3.289, 3814.398),
       (632.0, 2195.0, 0.0, 0.0, -37.6, 297.634, 3.298, 3341.125),
       (633.0, 2268.0, 0.0, 0.0, -45.747, -362.122, 3.299, 3533.769),
       (634.0, 2293.0, 0.0, 0.0, -55.147, -436.53, 3.307, 3490.382),
       (634.0, 2275.0, 0.0, 0.0, -55.773, -441.491, 3.308, 4010.713),
       (632.0, 1983.0, 0.0, 0.0, -27.628, -439.132, 3.313, 3593.826),
       (632.0, 1987.0, 0.0, 0.0, -27.628, -439.132, 3.314, 3860.664),
       (632.0, 1984.0, 0.0, 0.0, -27.628, -439.132, 3.314, 3645.711),
       (634.0, 2294.0, 0.0, 0.0, -55.773, -441.491, 3.316, 3482.873),
       (633.0, 2269.0, 0.0, 0.0, -45.747, -362.122, 3.32, 3511.671),
       (632.0, 1981.0, 0.0, 

In [20]:
results_to_cluster = results#filtered_results
arg = dict(eps=0.02, min_samples=1, n_jobs=-1)
clustered_results = ai.clusterResults(results_to_cluster, dbscan_args=arg)#, im_array, image_times)
clustered_results =  results_to_cluster[np.array(clustered_results[1], dtype=np.int)]
#best_targets = range(stamp_count)
#best_targets

In [21]:
print( len(clustered_results) )
clustered_results

48


array([(1969.0, 2838.0, 0.0, 0.0, 5.751, 0.0, 3.821, 214.793),
       (1181.0, 4053.0, 0.0, 0.0, 4.067, 4.067, 3.46, 473.178),
       (1517.0, 14.0, 0.0, 0.0, -2.627, -4.549, 2.753, 145.777),
       (1232.0, 4049.0, 0.0, 0.0, 5.235, 2.667, 2.675, 347.452),
       (1446.0, 13.0, 0.0, 0.0, 2.554, -5.014, 2.475, 137.78),
       (0.0, 429.0, 0.0, 0.0, -0.252, 0.311, 2.14, 106.699),
       (1720.0, 13.0, 0.0, 0.0, -1.877, -3.683, 2.001, 150.936),
       (413.0, 4056.0, 0.0, 0.0, 0.987, 0.265, 1.981, 128.25),
       (0.0, 1977.0, 0.0, 0.0, -0.262, 0.454, 1.864, 92.432),
       (1977.0, 816.0, 0.0, 0.0, 3.975, -1.527, 1.787, 86.756),
       (1986.0, 2084.0, 0.0, 0.0, 5.253, 2.016, 1.688, 214.7),
       (0.0, 2711.0, 0.0, 0.0, -0.325, -0.562, 1.642, 81.432),
       (5.0, 1023.0, 0.0, 0.0, -2.516, 2.794, 1.641, 123.852),
       (1369.0, 4054.0, 0.0, 0.0, -5.434, 0.861, 1.63, 105.294),
       (1527.0, 4052.0, 0.0, 0.0, 5.233, 1.7, 1.529, 188.503),
       (641.0, 0.0, 0.0, 0.0, 1.047, -0.467, 1.5

In [28]:
im_psi_array

array([[[  2.45735320e-14,   1.85936624e-14,  -3.06797936e-03, ...,
           4.71498146e-14,   4.21026908e-14,   1.72278810e-14],
        [  2.48269101e-14,   1.60374203e-14,  -3.06797959e-03, ...,
           4.08089158e-14,   2.79035574e-14,  -6.61686892e-15],
        [  2.56925591e-14,   1.56569805e-14,  -3.06797936e-03, ...,
           3.69866898e-14,   2.41691992e-14,  -8.76621502e-15],
        ..., 
        [ -2.13523842e-02,   2.86503928e-03,   6.90080831e-03, ...,
          -3.95019799e-02,  -5.00000156e-02,  -5.00000045e-02],
        [  7.58988736e-03,   2.69732792e-02,   3.38711925e-02, ...,
          -3.68773304e-02,  -5.00000119e-02,  -5.00000007e-02],
        [  5.94597822e-03,   3.48967388e-02,   4.30859439e-02, ...,
          -3.48459855e-02,  -5.00000045e-02,  -4.99999970e-02]],

       [[  8.73771589e-03,  -3.19584981e-02,  -3.30655165e-02, ...,
           3.63315782e-03,   2.45944504e-03,   8.35094193e-04],
        [ -2.92378608e-02,  -6.76216930e-02,  -6.65404350e-0

In [25]:
f_results = results#clustered_results
for imNum in range(min(len(f_results),50)):
    current = imNum#best_targets[imNum]    
    plt.imshow(ai.createPostageStamp(im_psi_array,
                                         list(f_results[['t0_x', 't0_y']][current]),
                                         np.array(list(f_results[['v_x', 'v_y']][current])),
                                         image_times, 
                                         [25., 25.])[0],
                    origin='lower',
               #cmap=plt.cm.Greys_r,
               interpolation='None')
    plt.title(str('#' + str(imNum+1) + ' [x,y] = ' + str(list(f_results[['t0_x', 't0_y']][current])))
                  + ' v = ' + str(list(f_results[['v_x', 'v_y']][current])))
 #   plt.savefig("stamps/stamp"+str(imNum+1)+".png")
    plt.show()


IndexError: index 4 is out of bounds for axis 0 with size 4